In [1]:
import re
import math
import nltk
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from sklearn.model_selection import train_test_split

In [2]:
with open ("31100.txt") as file:
    data= file.read()

In [3]:
sent_text = nltk.sent_tokenize(data) 

word_text = nltk.word_tokenize(data)


In [4]:
for i, item in enumerate(sent_text):
    s = item.replace("\n", " ")
    s = re.sub(r'[`~!@#$%^&*()+={}|\[\]:";<>?,\./“”]', '', s)
    s = '<s>' + " " + s + " " + "</s>"
    s = s.lower()
    sent_text[i] = ' '.join(s.split())

train, test= train_test_split(sent_text, test_size=0.2)

In [31]:
print(train[1])

<s> have you ever seen the lady </s>


In [6]:
def get_ngrams(text_lst, n ):
    
    count = {}
    total_count = 0
    for i in range(len(text_lst)):
        temp = text_lst[i].split()
        for j in range(len(temp)-n+1):
            
            temp2 = []
            for k in range(n):
                temp2.append(temp[j+k])
                s = " ".join(temp2)
            count[s] = count.get(s, 0) + 1
            total_count += 1
    return count, total_count

In [7]:
unigram, total_unigrams = get_ngrams(train, 1)
bigram, total_bigrams= get_ngrams(train, 2)
trigram, total_trigrams = get_ngrams(train, 3)
quadgram, total_quadgrams = get_ngrams(train, 4)

In [8]:
#MLE for unigram
MLE_unigrams = {}
for i in unigram.keys():
    MLE_unigrams[i] = unigram[i]/total_unigrams

#MLE for bigram
MLE_bigrams = {}
for i in bigram.keys():
    lst = i.split()
    MLE_bigrams[i] = bigram[i]/unigram[' '.join(lst[:len(lst)-1])]

#MLE for trigram
MLE_trigrams = {}
for i in trigram.keys():
    lst = i.split()
    MLE_trigrams[i] = trigram[i]/bigram[' '.join(lst[:len(lst)-1])]

#MLE for quadgram
MLE_quadgrams = {}
for i in quadgram.keys():
    lst = i.split()
    MLE_quadgrams[i] = quadgram[i]/trigram[' '.join(lst[:len(lst)-1])]

In [9]:
def Generator(n):
    if n == 1:
        probabilities = list(MLE_unigrams.values())
        sent_gen = '<s>'
        for i in range(15):
            lst = np.random.multinomial(1,probabilities).tolist()
            a = lst.index(1)
            keys = list(unigram.keys())
            chosenword = keys[a]
            if(chosenword == "<s>"):
                continue
            elif(chosenword == "</s>"):
                sent_gen = sent_gen + chosenword
                return 
            else:
                sent_gen = sent_gen + " " + chosenword             
        return sent_gen
    elif n == 2:
        probabilities = list(MLE_bigrams.values())
        sent_gen = '<s>'
        start = '<s>'
        wordlist = list(MLE_bigrams.keys())
        
        for i in range(15):
            NextWord = []
            prob = []
            for j in range(len(wordlist)):
                l = wordlist[j].split()
                if(l[0] == start):
                    NextWord.append(l[1])
                    prob.append(probabilities[j])
            
            for m in range(len(prob)):
                prob[m] = prob[m]/sum(prob)
            lst = np.random.multinomial(1, prob).tolist()
            a = lst.index(1)
            word = NextWord[a]
            if(word == "</s>"):
                sent_gen = sent_gen + word
                
                return
            else:
                sent_gen = sent_gen + " " + word
                start = word
        return sent_gen
    elif n == 3:
        probabilities_tri = list(MLE_trigrams.values())
        probabilities_bi = list(MLE_bigrams.values())
        wordlist = list(MLE_trigrams.keys())
        sent_gen = '<s>'
        start_bi = "<s>"
        NextWord_bi = []
        prob_bi = []
        bigrams = list(MLE_bigrams.keys())
        
        for i in range(len(bigrams)):
            l = bigrams[i].split()
            if(l[0] == start_bi):
                NextWord_bi.append(l[1])
                prob_bi.append(probabilities_bi[i])
        lst_bi = np.random.multinomial(1, prob_bi).tolist()
        a = lst_bi.index(1)
        
        word_bi = NextWord_bi[a]   
        list_bi = [start_bi, word_bi]
        
        sent_gen = sent_gen +  " " + word_bi 
        for i in range(15):
            NextWord_tri = []
            prob_tri = []
            for j in range(len(wordlist)):
                l = wordlist[j].split()
                if(l[0:2] == list_bi):
                    NextWord_tri.append(l[2])
                    prob_tri.append(probabilities_tri[j])
            for m in range(len(prob_tri)):
                prob_tri[m] = prob_tri[m]/sum(prob_tri)
            
            lst_tri = np.random.multinomial(1, prob_tri).tolist()
            b = lst_tri.index(1)
            
            word_tri = NextWord_tri[b]
            list_tri = [list_bi[1],word_tri]
            if(word_tri == "</s>"):
                sent_gen = sent_gen + word_tri
                print(sent_gen, end ="")
                return
            else:
                sent_gen = sent_gen + " " + word_tri
        return sent_gen
    elif n == 4:
        probabilities_tri = list(MLE_trigrams.values())
        probabilities_bi = list(MLE_bigrams.values())
        probabilities_quad = list(MLE_quadgrams.values())
        wordlist = list(MLE_quadgrams.keys())
        sentence = "<s> "
        start_bi = "<s>"
        nextWord_bi = []
        prob_bi = []
        bigrams = list(MLE_bigrams.keys())
        trigrams = list(MLE_trigrams.keys())
        
        for i in range(len(bigrams)):
            l = bigrams[i].split()
            if(l[0] == start_bi):
                nextWord_bi.append(l[1])
                prob_bi.append(probabilities_bi[i])
        chosenarray_bi = np.random.multinomial(1, prob_bi)
        chosenword_bi = nextWord_bi[int(np.where(chosenarray_bi == 1)[0])]
        bigramTuple = (start_bi, chosenword_bi)
        sentence = sentence + chosenword_bi + " "
        nextWord_tri = []
        prob_tri = []
        for i in range(len(trigrams)):
            l = trigrams[i].split()
            if(l[0] == bigramTuple[0] and l[1] == bigramTuple[1]):
                nextWord_tri.append(l[2])
                prob_tri.append(probabilities_tri[i])
        chosenarray_tri = np.random.multinomial(1, prob_tri)
        chosenword_tri = nextWord_tri[int(np.where(chosenarray_tri == 1)[0])]
        trigramTuple = (start_bi, chosenword_bi,chosenword_tri)
        sentence = sentence + chosenword_tri + " "
        for j in range(15):
            nextWord_quad = []
            prob_quad = []
            for i in range(len(wordlist)):
                l = wordlist[i].split()
                if(l[0] == trigramTuple[0] and l[1] == trigramTuple[1] and l[2] == trigramTuple[2]):
                    nextWord_quad.append(l[3])
                    prob_quad.append(probabilities_quad[i])
           
            chosenarray_quad = np.random.multinomial(1, prob_quad)
            chosenword_quad = nextWord_quad[int(np.where(chosenarray_quad == 1)[0])]
            trigramTuple = (trigramTuple[1],trigramTuple[2], chosenword_quad)
          
            if(chosenword_quad == "</s>"):
                sentence = sentence + chosenword_quad
                print(sentence, end ="")
                return
            else:
                sentence = sentence + chosenword_quad + " "


In [44]:
print(Generator(4))

<s> but you must come indeed </s>None


In [45]:
for _ in range(5):
    print(Generator(1))

None
<s> part i hope by of know behind could happiness her a above worse or deal
<s> there lower all you you of large park and to the both continued a cottage
None
<s> obliged about do for the patience our is fresh sudden lady seen the of whole


In [43]:
for _ in range(5):
    print(Generator(2))

None
None
<s> yes she calmly begged leave a quivering lip emma thought elinor told me to be
<s> she was likely to cloud when the objects as to such varieties of meeting with
<s> the others still was a joke with captain tilney as such inhumanity and face you


In [13]:
for _ in range(5):
    print(Generator(3))

<s> the history kellynch apothecary entrance crawfords wants boy beautifull next letter cottage event first comfort holidays
<s> we dressed are always shall are had lived must are have happened want perfectly have must
<s> no ma'am one more i no--do no my she i hurry theatre i you i my
<s> i ran like do venture hope perfectly consider have did am cannot dare thought had wish
<s> in nursing half this a the one one seeing general the no his a this all


In [14]:
for _ in range(5):
    print(Generator(4))

<s> it may not be for several years </s>None
<s> and moreover if you must go you cannot refuse to come </s>None
None
<s> it is a shocking trick for a young man like mr allen </s>None
None


In [40]:
def ProbUnigram(word_list):
    log_sum_uni = 0 
    vals= list(MLE_unigrams.values())
    keys = list(MLE_unigrams.keys())
    for i in word_list:
        index = keys.index(i)
        try:
            log_sum_uni += math.log(vals[index])
        except:
            log_sum_uni+=0
    print("Unigram Probability=",math.exp(log_sum_uni))
    print("Perplexity for Unigram=",1/(math.exp(log_sum_uni))**(1/len(word_list)))
    
def ProbBigram(word_list):
    log_sum_bi = 0 
    vals= list(MLE_bigrams.values())
    keys = list(MLE_bigrams.keys())
    for i in word_list:
        index = keys.index(i)
        try:
            log_sum_bi += math.log(vals[index])
        except:
            log_sum_bi+=0
    print("Bigram Probability=",math.exp(log_sum_bi))
    print("Perplexity for Bigram=",1/(math.exp(log_sum_bi))**(1/len(word_list)))
    
def ProbTrigram(word_list):
    log_sum_tri = 0 
    vals= list(MLE_trigrams.values())
    keys = list(MLE_trigrams.keys())
    for i in word_list:
        index = keys.index(i)
        try:
            log_sum_tri += math.log(vals[index])
        except:
            log_sum_tri+=0
    print("Trigram probability =",math.exp(log_sum_tri))
    print("Perplexity for Trigram=",1/(math.exp(log_sum_tri))**(1/len(word_list)))
def ProbQuadgram(word_list):
    log_sum_quad = 0 
    vals= list(MLE_trigrams.values())
    keys = list(MLE_trigrams.keys())
    for i in word_list:
        index = keys.index(i)
        try:
            log_sum_quad += math.log(vals[index])
        except:
            log_sum_quad+=0
    print("Log space probability Quadgram =",math.exp(log_sum_quad))
    print("Perplexity for Quadgram=",1/(math.exp(log_sum_quad))**(1/len(word_list)))
    
    
def Probability(sentence,model_name):
    sentence = sentence.replace("\n"," ").replace("\r"," ").replace("’","").replace("-"," ")
    sentence = re.sub('[^ A-Za-z0-9]',' ',sentence)
    sentence = sentence.lower()
    words_list =[] 
    words_list.append("<s>")
    words_list.extend(word_tokenize(sentence))
    words_list.append("</s>")
    bigrams = list(ngrams(words_list,2))
    trigrams = list(ngrams(words_list,3))
    quadgrams = list(ngrams(words_list,4))
   
    for i in range(len(bigrams)):
        bigrams[i] = bigrams[i][0]+ " " +bigrams[i][1]
   
    for i in range(len(trigrams)):
        trigrams[i] = trigrams[i][0]+ " " +trigrams[i][1] + " "+ trigrams[i][2]
    
    for i in range(len(quadgrams)):
        quadgrams[i] = quadgrams[i][0]+ " " +quadgrams[i][1] + " " + quadgrams[i][2]+ " " +quadgrams[i][3]
 
        
    if(model_name == "Unigram"):
        ProbUnigram(words_list)
    elif(model_name == "Bigram"):
        ProbBigram(bigrams)
    elif(model_name == "Trigram"):
        ProbTrigram(trigrams)
    elif(model_name == "Quadgram"):
        ProbQuadgram(quadgrams)


sentence = "But now, another occupation and solicitude of mind was beginning to be added to these."
Probability(sentence, "Unigram")
Probability(sentence, "Bigram")
Probability(sentence, "Trigram")




Unigram Probability= 3.086215246700652e-44
Perplexity for Unigram= 362.6147926617235
Bigram Probability= 8.883107071948704e-31
Perplexity for Bigram= 75.54656006026416
Trigram probability = 8.173503478675116e-16
Perplexity for Trigram= 10.135366319325014
